In [9]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split as tts
from transformers import ViTForImageClassification, ViTFeatureExtractor, ViTConfig
import torch.nn.functional as F

In [10]:
resnet_model = models.resnet18(pretrained=False)
resnet_model.fc = torch.nn.Linear(resnet_model.fc.in_features, 2)    # Adjust the final layer for binary classification
resnet_model.load_state_dict(torch.load('../ammad/best_model.pth', map_location=torch.device('cpu')), strict=False)

vit_model = ViTConfig(num_labels=2)  # Binary classification
vit_model = ViTForImageClassification(vit_model)
vit_model.load_state_dict(torch.load(os.path.join("vision-transformer-models", "model9.pth"), map_location=torch.device('cpu')))

<All keys matched successfully>

In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = ImageFolder('data/original-dataset', transform=transform)

train_dataset, test_dataset = tts(dataset, test_size=0.2)

In [12]:
len(train_dataset), len(test_dataset)

(752, 188)

In [13]:
dataset.class_to_idx

{'CANCER': 0, 'NON_CANCER': 1}

In [14]:
folder_path = "data/NON_CANCER(Augmented)"

image_files = [f for f in os.listdir(folder_path)]

augmented_images = []
for image_file in image_files:
    img_path = os.path.join(folder_path, image_file)
    img = Image.open(img_path)
    augmented_images.append((transform(img), 1))
    
train_dataset += augmented_images

In [18]:
def soft_voting(data, models):
    count = 0
    labels = []
    preds = []
    for image in data:
        with torch.no_grad():
            resnet_output = models[0](image[0].unsqueeze(0))
            vision_transformer_output = models[1](image[0].unsqueeze(0)).logits
            resnet_probs = F.softmax(resnet_output, dim=1).numpy()
            vision_transformer_probs = F.softmax(vision_transformer_output, dim=1).numpy()
            
            # Averaging probabilities
            avg_probs = (resnet_probs + vision_transformer_probs) / 2
            final_pred = np.argmax(avg_probs, axis=1)
            
            if image[1] == final_pred:
                count += 1
                
            labels.append(image[1])
            preds.append(final_pred)
            
                
    return count / len(data), labels, preds

In [19]:
accuracy, labels, preds = soft_voting(test_dataset, [resnet_model, vit_model])

In [20]:
from sklearn.metrics import f1_score
f1 = f1_score(labels, preds, average='binary')

In [21]:
print(accuracy, f1)

0.9840425531914894 0.9837837837837838
